<a href="https://colab.research.google.com/github/felzr/Android-GoogleMaps-Firebase/blob/master/contextual_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [182]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [183]:
import numpy as np
import tflearn
import tensorflow as tf
import random

In [184]:
import json
with open('intents-base.json') as json_data:
    intents = json.load(json_data)

In [185]:
words = []
classes = []
documents = []
ignore_words = ['?']

In [186]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [187]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [188]:
classes = sorted(list(set(classes)))

In [189]:
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

29 documents
10 classes ['aberto hoje', 'adeus', 'aluguel', 'final de semana', 'hoje', 'horas', 'motos', 'obrigado', 'pagamentos', 'saudação']
48 unique stemmed words ['aberto', 'abr', 'aceitam', 'alguém', 'alug', 'apena', 'até', 'aí', 'bom', 'cartõ', 'ciclomot', 'como', 'crédito', 'de', 'dia', 'dinheiro', 'está', 'existem', 'fds', 'fin', 'funcion', 'gostar', 'hoj', 'hora', 'horário', 'isso', 'logo', 'mastercard', 'moto', 'o', 'obrigado', 'oi', 'olá', 'podemo', 'qua', 'qual', 'quando', 'que', 'seman', 'seu', 'tchau', 'tem', 'tipo', 'um', 'vai', 'você', 'é', 'útil']


In [190]:
training = []
output = []

In [191]:
output_empty = [0] * len(classes)

In [192]:
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [193]:
random.shuffle(training)
training = np.array(training)

<ipython-input-193-cc0bc1dae1db>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [194]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [195]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

In [196]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=500, batch_size=8, show_metric=True)

Training Step: 1999  | total loss: 0.12345 | time: 0.016s
| Adam | epoch: 500 | loss: 0.12345 - acc: 0.9988 -- iter: 24/29
Training Step: 2000  | total loss: 0.11880 | time: 0.023s
| Adam | epoch: 500 | loss: 0.11880 - acc: 0.9990 -- iter: 29/29
--


In [197]:
model.save('model.tflearn')

In [198]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [199]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [200]:
import json
with open('intents-base.json') as json_data:
    intents = json.load(json_data)

In [201]:
model.load('./model.tflearn')

In [202]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [203]:
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [204]:
p = bow("Você está aberto hoje??", words)
print (p)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0]


In [205]:
context = {}
ERROR_THRESHOLD = 0.25

In [206]:
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [207]:
def response(sentence, userID='Bastiao', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [209]:
response('Podemos alugar um ciclomotor?')
response('fds')

Você pretende alugar hoje ou no final desta semana?
Para no final de semena ligue 9999999999999, no fds ligue para 1-800-MYMOPED


In [ ]:
classify('hoje')

[('hoje', 0.9849037)]

In [210]:
context

{'Bastiao': 'dia_aluguel'}